# Super Resolution GAN (SRGAN) implementation

### Import needed modules

In [1]:
from torch import nn

### Building blocks

In [ ]:
class ConvBlock(nn.Module):
    # Conv -> BN (Batch Norm) -> Leaky/PReLu
    def __init__(
        self,
        in_channels,
        out_channels,
        discriminator=False,
        use_act=True,
        use_bn=True,
        **kwargs,
    ):
        super().__init__()
        self.use_act = use_act
        self.cnn = nn.Conv2d(in_channels, out_channels, **kwargs, bias=not use_bn)
        self.bn = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()
        self.act = (
            nn.LeakyReLU(0.2, inplace=True) if discriminator else nn.PReLU(num_parameters=out_channels)
        )
    
    def forward(self, x):
        return self.act(self.bn(self.cnn(x))) if self.use_act else self.bn(self.cnn(x))

class UpsampleBlock(nn.Module):
    def __init__(self, in_c, scale_factor):
        super().__init__()
        
        self.conv = nn.Conv2d(in_c, in_c * scale_factor ** 2, 3, 1, 1)
    pass

class ResidualBlock(nn.Module):
     pass

### Generator

### Discriminator